In [359]:
import numpy as np
import scipy as sp
import pandas as pd
import random
import matplotlib.pyplot as plt
import  networkx as nx
import tensorflow as tf

%matplotlib inline

In [360]:
def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=1.0, p_out=0.0):
    #gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out)
    A = nx.adjacency_matrix(G).todense()
    
    return A


communities = 2 #number of communities, chance to 
group_size = 6 #number of nodes in each communitites (balanced so far)
dim_graph = communities*group_size
A = np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.2)).astype(np.double)

data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.2)).astype(np.float64) for i in range(100)]
data[1]

array([[ 0.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.]])

In [361]:
communities = 6
Adj = tf.placeholder(tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

r =  tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=2.0, dtype=tf.float64,
                                 seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([communities], dtype=tf.float64),
                                      tf.ones([communities], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([communities], dtype=tf.float64),
                                      tf.zeros([communities], dtype=tf.float64)])


y_a = tf.pack([true_assignment_a, true_assignment_b]) 
y_b = tf.pack([true_assignment_b, true_assignment_a]) 


eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])

centers = tf.Variable(tf.random_normal(shape=[2,2],
                                       mean=0.0, stddev=1.0, dtype=tf.float64,seed=None, name=None))

expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
expanded_centers = tf.expand_dims(centers, 1)

distances = tf.transpose(tf.reduce_sum(tf.square(tf.sub(expanded_vectors,
                                                        expanded_centers)), 2))

assignments = tf.nn.softmax(distances)

loss_vec_a = y_a*tf.log(tf.transpose(assignments))
loss_vec_b = y_b*tf.log(tf.transpose(assignments))

loss = tf.minimum(tf.reduce_sum(loss_vec_a), tf.reduce_sum(loss_vec_a))



optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r, centers])

with tf.Session() as sess:
    for i in range(10):
        sess.run(init)
#       sess.run(train)
        print sess.run([r, loss, centers], feed_dict = {Adj: data[i]})
    
    





[array([-2.29422856]), -28.218322145527342, array([[ 2.24012646,  0.42207945],
       [ 0.26281016,  0.65686633]])]
[array([ 3.48630202]), -9.4914547900525719, array([[-0.72413713,  0.56843265],
       [-1.07142793,  0.27965509]])]
[array([ 1.55147772]), -9.6775516518713474, array([[-0.60250776,  0.1418508 ],
       [ 0.54717446, -0.71255472]])]
[array([ 3.66483497]), -10.564502378297023, array([[-1.34909143,  0.721417  ],
       [-1.53775463,  0.07023038]])]
[array([ 0.1531855]), -43.464667764348576, array([[ 0.91999059, -0.71303387],
       [-2.7090699 ,  0.4627907 ]])]
[array([ 0.52446018]), -8.3523408007056883, array([[ 0.5934189 ,  1.65400852],
       [ 1.46903016, -0.59488227]])]
[array([-1.88918572]), -10.657993620307419, array([[-0.2882252 , -0.92844668],
       [-1.28110541,  0.71369545]])]
[array([ 1.27789765]), -9.4241652821622992, array([[ 1.07879068, -0.22778099],
       [ 0.18856706, -0.33987884]])]
[array([ 5.60640361]), -15.795405685882415, array([[ 0.68467053, -1.55385

In [362]:
#now with training uncommentedout

In [364]:
communities = 6
Adj = tf.placeholder(tf.float64)
Adj = tf.cast(Adj, tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

#r =  tf.Variable(tf.constant(2.1), dtype = tf.float64)
r = tf.Variable(tf.random_normal(shape=[1], mean=0.0, stddev=2.0, dtype=tf.float64,seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([communities], dtype=tf.float64),
                                      tf.ones([communities], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([communities], dtype=tf.float64),
                                      tf.zeros([communities], dtype=tf.float64)])

y_a = tf.pack([true_assignment_a, true_assignment_b]) 

eigenvec_proj = tf.nn.softmax(tf.slice(eigenvec, [0,0], [dim_graph, 2]))
eigenvec_proj = tf.cast(tf.reduce_sum(eigenvec, 1), dtype = tf.float64)


loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))), 
              tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))
#tf.reduce_sum(tf.square(tf.sub(c, true_assignment_a)))


optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(loss, var_list=[r])

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r])

with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        sess.run(train, feed_dict = {Adj: data[i]})
        print sess.run([r, loss_grad, eigenvec_grad, loss, y_a], feed_dict = {Adj: data[i]})
    
    



[array([ 0.98725889]), [array([ 5.78273031])], [array([-2.60173469])], 77.000566548209179, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([ 0.48435297]), [array([-12.55051038])], [array([ 8.7954016])], 81.349320431339464, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([-0.06340231]), [array([ 6.00621583])], [array([ 1.01199955])], 76.306640524838542, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([-0.44877546]), [array([ 11.41532667])], [array([-2.13210363])], 78.405018226548265, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([-0.35192856]), [array([ 0.17654144])], [array([ 3.08925243])], 78.6

In [365]:

tf.reset_default_graph()

In [366]:
?tf.pack


In [367]:
tw = tf.train.SummaryWriter('./logDir',sess.graph)

# Train your network

tw.flush()
tw.close()

In [368]:
communities = 2 #number of communities, chance to 
group_size = 5
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.8, p_out=0.3)).astype(np.float64) for i in range(1000)]
data[1]

array([[ 0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  0.],
       [ 1.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 1.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.]])

In [369]:
communities = 2
group_size=5
dim_graph=communities*group_size

Adj = tf.placeholder(tf.float64)
Adj = tf.cast(Adj, tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

average_degree = tf.reduce_sum(Adj)/dim_graph

#r =  tf.Variable(tf.constant(2.1), dtype = tf.float64)
r = tf.Variable(tf.random_normal(shape=[1], mean=0.0, stddev=2.0, dtype=tf.float64,seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([group_size], dtype=tf.float64),
                                      tf.zeros([group_size], dtype=tf.float64)])

y_a = tf.pack([true_assignment_a, true_assignment_b]) 
y_b = tf.pack([true_assignment_b, true_assignment_a]) 


eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])


center = tf.Variable(tf.truncated_normal(shape=[2,2],
                                       mean=0.0, stddev=0.6, dtype=tf.float64,seed=None, name=None))

expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
expanded_centers = tf.expand_dims(center, 1)

distances = tf.reduce_sum(tf.square(tf.sub(expanded_vectors, expanded_centers)), 2)

hard_assignment = tf.argmin(distances, 0)
new_centers = tf.concat(0, [tf.reduce_mean(tf.gather(eigenvec_proj,
                                                     tf.reshape(tf.where(tf.equal(hard_assignment, c)),
                                                                [1,-1])), reduction_indices=[1]) for c in xrange(communities)])

#a = tf.slice(new_centers, [0,0], [1,2])
#b = tf.slice(new_centers, [1,0], [1,2])
update_centers = tf.assign(center, new_centers)


assignments = 1-tf.nn.softmax(tf.transpose(distances)) #this ensures that the smaller distances are actually the correct assignments as being far from the centers make it less likely it is in that cluster
#warning, the above transform will definitely not work for more than 2 centers, so need to be more clever later...
#like basically sum the other two entries and subtract, this will be the generalized algo.... 


loss_vec_a = y_a*tf.log(tf.transpose(assignments))
loss_vec_b = y_b*tf.log(tf.transpose(assignments))

loss = tf.minimum(-tf.reduce_sum(loss_vec_a), -tf.reduce_sum(loss_vec_b))


#loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))),tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))


optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss, var_list=[r])

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r, center])

with tf.Session() as sess:
    sess.run(init)
    sess.run(train, feed_dict = {Adj: data[i]})
    for i in range(100):
        sess.run([train], feed_dict = {Adj: data[i]})
        if i%10 ==0:
            print sess.run([loss, hard_assignment, r, average_degree, center, new_centers], feed_dict = {Adj: data[i]})
    
#, distances, assignments,  loss_grad, eigenvec_grad, 



[7.6475098485959974, array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1]), array([-2.22711424]), 5.4000000000000004, array([[ 0.339894  , -0.11127823],
       [ 0.2946393 , -0.8380243 ]]), array([[-0.01925924,  0.15732062],
       [-0.06024371, -0.48703915]])]
[7.5743741607489232, array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0]), array([-2.22086506]), 4.5999999999999996, array([[ 0.339894  , -0.11127823],
       [ 0.2946393 , -0.8380243 ]]), array([[ 0.01525243,  0.0970721 ],
       [-0.21208723, -0.53004248]])]
[7.554480037746468, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([-2.10880414]), 4.4000000000000004, array([[ 0.339894  , -0.11127823],
       [ 0.2946393 , -0.8380243 ]]), array([[-0.00222552,  0.04211762],
       [        nan,         nan]])]
[7.7563432199324636, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([-1.89925314]), 5.2000000000000002, array([[ 0.339894  , -0.11127823],
       [ 0.2946393 , -0.8380243 ]]), array([[-0.00718779,  0.05680489],
       [        nan,         nan]])]
[7.23848338521408

In [370]:

tf.reset_default_graph()

In [371]:
#does backprop not play well with update centers? 
?tf.assign


In [372]:


center_1 = tf.Variable(tf.random_normal(shape=[2,1],
                                       mean=1.0, stddev=1.0, dtype=tf.float64,seed=None, name=None))
center_2 = tf.Variable(tf.random_normal(shape=[2,1],
                                       mean=-1.0, stddev=1.0, dtype=tf.float64,seed=None, name=None))
center = tf.pack([center_1, center_2])

In [373]:

from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import linalg_ops
from tensorflow.python.ops import math_ops


In [374]:

@ops.RegisterGradient("aj")
def GradientTest2(op, grad_e, grad_v):
    """Gradient for SelfAdjointEigV2 derived with Joan with grad_stiefel instead of grassman"""
    e = op.outputs[0]
    v = op.outputs[1]
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            E = array_ops.diag(e)
            #v_proj = arrary.ops.slice(v, [0,0], [])
            grad_stiefel = grad_v - math_ops.batch_matmul(math_ops.batch_matmul(v, grad_v), array_ops.transpose(v))
            grad_a = math_ops.batch_matmul(E, math_ops.batch_matmul(v, array_ops.transpose(grad_stiefel))
                                           +math_ops.batch_matmul(grad_stiefel, array_ops.transpose(v)))
    return grad_a 

KeyError: "Registering two gradient with name 'aj' !(Previous registration was in <module> /Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3)"

In [377]:
communities = 2
group_size=10
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.9, p_out=0.0)).astype(np.float64) for i in range(1000)]




dim_graph=communities*group_size

Adj = tf.placeholder(tf.float64)
Adj = tf.cast(Adj, tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

average_degree_root = tf.sqrt(tf.reduce_sum(Adj)/dim_graph)

#r =  tf.Variable(tf.constant(2.1), dtype = tf.float64)
r = tf.Variable(tf.truncated_normal(shape=[1], mean=2.0, stddev=0.05, dtype=tf.float64,seed=None, name=None))

Bethe_Hesse_neg = (r*r-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 


with tf.Session() as sess:
    g = tf.get_default_graph()
    with g.gradient_override_map({'SelfAdjointEigV2': 'aj'}):

        eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

        true_assignment_a = tf.concat(0, [tf.zeros([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
        true_assignment_b = tf.concat(0, [tf.ones([group_size], dtype=tf.float64),
                                      tf.zeros([group_size], dtype=tf.float64)])

        y_a = tf.pack([true_assignment_a, true_assignment_b]) 
        y_b = tf.pack([true_assignment_b, true_assignment_a]) 


        eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])


        #center = # tf.Variable(tf.slice(tf.random_shuffle(eigenvec_proj),[0,0],[2,-1]))
        center = tf.Variable(tf.random_normal(shape=[2,2],
                                       mean=0.0, stddev=0.2, dtype=tf.float64,seed=None, name=None))

        expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
        expanded_centers = tf.expand_dims(center, 1)

        distances = tf.reduce_sum(tf.square(tf.sub(expanded_vectors, expanded_centers)), 2)

        hard_assignment = tf.argmin(distances, 0)
        new_centers = tf.concat(0, [tf.reduce_mean(tf.gather(eigenvec_proj,
                                                     tf.reshape(tf.where(tf.equal(hard_assignment, c)),
                                                                [1,-1])), reduction_indices=[1]) for c in xrange(communities)])

#a = tf.slice(new_centers, [0,0], [1,2])
#b = tf.slice(new_centers, [1,0], [1,2])
        a = tf.shape(center)
        b = tf.shape(new_centers)
        update_centers = tf.assign(center, new_centers)


        assignments = 1-tf.nn.softmax(tf.transpose(distances)) #this ensures that the smaller distances are actually the correct assignments as being far from the centers make it less likely it is in that cluster
#warning, the above transform will definitely not work for more than 2 centers, so need to be more clever later...
#like basically sum the other two entries and subtract, this will be the generalized algo.... 

      
        loss_vec_a = y_a*tf.log(tf.transpose(assignments))
        loss_vec_b = y_b*tf.log(tf.transpose(assignments))

        loss = tf.minimum(-tf.reduce_sum(loss_vec_a), -tf.reduce_sum(loss_vec_b))


#loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))),tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))


        optimizer = tf.train.GradientDescentOptimizer(0.5)
        train = optimizer.minimize(loss, var_list=[r])

        eigenvec_grad = tf.gradients(eigenvec, r)
        loss_grad = tf.gradients(loss, r)

        init = tf.initialize_variables([r, center])

        init_center = tf.initialize_variables([center])
        sess.run(init)
        for i in range(10):
            sess.run([train], feed_dict = {Adj: data[i]})
            if i%1 ==0:
                print i
                print sess.run([loss_grad, loss, hard_assignment, r], feed_dict = {Adj: data[i]})
    
#, distances, assignments,  loss_grad, eigenvec_grad, 



0
[[array([ 31206.6603255])], 13.86057974086817, array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0]), array([-108.20818198])]
1
[[array([  4.41502137e+08])], 12.823165429285037, array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([ 5728.01548279])]
2
[[array([  8.19789132e+16])], 13.805720539663744, array([1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0]), array([ -2.84980941e+08])]
3
[[array([  1.59894950e+36])], 13.829488268276783, array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]), array([  1.78423974e+17])]
4
[[array([ -3.83297014e+91])], 13.524456998121043, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([ -7.81603036e+35])]
5
[[array([  9.04092829e+258])], 13.899074058700997, array([0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([  3.83297014e+91])]
6


InvalidArgumentError: Self Adjoint Eigen decomposition was not successful. The input might not be valid.
	 [[Node: SelfAdjointEigV2_2 = SelfAdjointEigV2[T=DT_DOUBLE, _gradient_op_type="aj", compute_v=true, _device="/job:localhost/replica:0/task:0/cpu:0"](add_2)]]
Caused by op u'SelfAdjointEigV2_2', defined at:
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/xiangli/anaconda/lib

In [304]:

tf.reset_default_graph()

In [356]:

def Bethe_Hessian_NN(communities, group_size, data):

    dim_graph=communities*group_size

    Adj = tf.placeholder(tf.float64)
    Adj = tf.cast(Adj, tf.float64)

    Diag = tf.diag(tf.reduce_sum(Adj,0))
    Diag = tf.cast(Diag, tf.float64)

    average_degree_root = tf.sqrt(tf.reduce_sum(Adj)/dim_graph)


    r = tf.Variable(tf.truncated_normal(shape=[1], mean=2.0, stddev=0.05, dtype=tf.float64,seed=None, name=None))

    Bethe_Hesse_neg = (r*r-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

    eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

    true_assignment_a = tf.concat(0, [tf.zeros([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
    true_assignment_b = tf.concat(0, [tf.ones([group_size], dtype=tf.float64),
                                      tf.zeros([group_size], dtype=tf.float64)])

    y_a = tf.pack([true_assignment_a, true_assignment_b]) 
    y_b = tf.pack([true_assignment_b, true_assignment_a]) 


    eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])

    
    #center = tf.Variable(tf.slice(tf.random_shuffle(eigenvec_proj),[0,0],[communities,-1]))
    
    center = tf.Variable(tf.random_normal(shape=[2,2],
                                       mean=0.0, stddev=0.1, dtype=tf.float64,seed=None, name=None))

    expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
    expanded_centers = tf.expand_dims(center, 1)

    distances = tf.reduce_sum(tf.square(tf.sub(expanded_vectors, expanded_centers)), 2)

    hard_assignment = tf.argmin(distances, 0)
    new_centers = tf.concat(0, [tf.reduce_mean(tf.gather(eigenvec_proj,
                                                     tf.reshape(tf.where(tf.equal(hard_assignment, c)),
                                                                [1,-1])), reduction_indices=[1]) for c in xrange(communities)])

    update_centers = tf.assign(center, new_centers)


    assignments = 1-tf.nn.softmax(tf.transpose(distances)) 

    loss_vec_a = y_a*tf.log(tf.transpose(assignments))
    loss_vec_b = y_b*tf.log(tf.transpose(assignments))

    loss = tf.minimum(-tf.reduce_sum(loss_vec_a), -tf.reduce_sum(loss_vec_b))


    optimizer = tf.train.GradientDescentOptimizer(0.5)
    train = optimizer.minimize(loss, var_list=[r])

    eigenvec_grad = tf.gradients(eigenvec, r)
    loss_grad = tf.gradients(loss, r)

    init = tf.initialize_variables([r, center])

    init_center = tf.initialize_variables([center])

    with tf.Session() as sess:
        sess.run(init)
        for i in range(100):
            sess.run([train, init_center], feed_dict = {Adj: data[i]})
            if i%2 ==0:
                
                print i
                print sess.run([loss, loss_grad, eigenvec_grad, hard_assignment, r, average_degree_root, new_centers], feed_dict = {Adj: data[i]})
    
#, distances, assignments,  loss_grad, eigenvec_grad, 



In [357]:
communities = 2
group_size=10
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.4, p_out=0.1)).astype(np.float64) for i in range(1000)]


Bethe_Hessian_NN(2, 10, data)

0
[13.281593850044475, [array([ 0.02381656])], [array([-0.31518388])], array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0]), array([ 2.07848215]), 2.0976176963403033, array([[-0.22258405, -0.15018963],
       [-0.20497194,  0.24865033]])]
2
[13.866828775315692, [array([ 0.01650381])], [array([ 0.69533327])], array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([ 2.07169177]), 2.0976176963403033, array([[-0.21349675, -0.01661341],
       [        nan,         nan]])]
4
[12.809980467669877, [array([ 0.01949234])], [array([ 0.9331938])], array([0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([ 2.06889919]), 2.3237900077244502, array([[-0.21332163,  0.09340305],
       [-0.23134388, -0.18724964]])]
6
[13.619542598693972, [array([-0.0139223])], [array([ 0.56111088])], array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]), array([ 2.07867274]), 2.1447610589527217, array([[-0.2200739 ,  0.26970118],
       [-0.21815849, -0

In [245]:

def EigenDecomp_r_diff(iterations, communities, group_size, data, r1, r2):

    dim_graph=communities*group_size

    Adj = tf.placeholder(tf.float64)
    Adj = tf.cast(Adj, tf.float64)

    Diag = tf.diag(tf.reduce_sum(Adj,0))
    Diag = tf.cast(Diag, tf.float64)

    average_degree_root = tf.sqrt(tf.reduce_sum(Adj)/dim_graph)
    
    r1 = tf.cast(r1, dtype=tf.float64)
    r2 = tf.cast(r2, dtype=tf.float64)
    
    def BH(r):
        return (r*r-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 



    BH1 = BH(r1)
    BH2 = BH(r2)

    eigenval1, eigenvec1 = tf.self_adjoint_eig(BH1)
    eigenval2, eigenvec2 = tf.self_adjoint_eig(BH2)
    
    e1vec_norm = tf.sqrt(tf.reduce_sum(tf.matmul(eigenvec1, eigenvec1)))
    e2vec_norm = tf.sqrt(tf.reduce_sum(tf.matmul(eigenvec2, eigenvec2)))
    
    operator_diff = tf.reduce_sum(tf.matmul(eigenvec1, eigenvec2))/(e1vec_norm*e2vec_norm)
    
    e1_norm = tf.sqrt(tf.reduce_sum(eigenval1*eigenval1))
    e2_norm = tf.sqrt(tf.reduce_sum(eigenval2*eigenval2))
    eigenval_dot = tf.reduce_sum(eigenval1*eigenval2)/(e1_norm*e2_norm)

    with tf.Session() as sess:
        for i in range(iterations):
            print i
            print sess.run([average_degree_root, eigenval_dot, operator_diff], feed_dict = {Adj: data[i]})


def EigenDecomp_r_diff_proj(iterations, communities, group_size, data, r1, r2, project_dim):

    dim_graph=communities*group_size

    Adj = tf.placeholder(tf.float64)
    Adj = tf.cast(Adj, tf.float64)

    Diag = tf.diag(tf.reduce_sum(Adj,0))
    Diag = tf.cast(Diag, tf.float64)

    average_degree_root = tf.sqrt(tf.reduce_sum(Adj)/dim_graph)
    
    r1 = tf.cast(r1, dtype=tf.float64)
    r2 = tf.cast(r2, dtype=tf.float64)
    
    def BH(r):
        return (r*r-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 


    BH1 = BH(r1)
    BH2 = BH(r2)

    eigenval1, eigenvec1 = tf.self_adjoint_eig(BH1)
    eigenval2, eigenvec2 = tf.self_adjoint_eig(BH2)
    
    eigenvec1 = tf.slice(eigenvec1, [0,0], [dim_graph, project_dim])
    eigenvec2 = tf.slice(eigenvec2, [0,0], [dim_graph, project_dim])
    
    e1vec_norm = tf.sqrt(tf.reduce_sum(tf.matmul(eigenvec1, tf.transpose(eigenvec1))))
    e2vec_norm = tf.sqrt(tf.reduce_sum(tf.matmul(eigenvec2, tf.transpose(eigenvec2))))
    
    operator_diff = tf.reduce_sum(tf.matmul(eigenvec1, tf.transpose(eigenvec2)))/(e1vec_norm*e2vec_norm)
    
    e1_norm = tf.sqrt(tf.reduce_sum(eigenval1*eigenval1))
    e2_norm = tf.sqrt(tf.reduce_sum(eigenval2*eigenval2))
    eigenval_dot = tf.reduce_sum(eigenval1*eigenval2)/(e1_norm*e2_norm)

    lst = []
    
    with tf.Session() as sess:
        for i in range(iterations):
            
            lst.append(sess.run([average_degree_root, eigenval_dot, operator_diff], feed_dict = {Adj: data[i]}))
    return np.asarray(lst)
            
  


In [221]:
communities = 2
group_size=5
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.2)).astype(np.float64) for i in range(1000)]


EigenDecomp_r_diff(100, 2, 5, data, 4, 4.5)

0
[1.3416407864998738, 0.99950496573195446, nan]
1
[1.8973665961010275, 0.9995177687259964, 0.13649340617465697]
2
[1.4142135623730951, 0.99947153734202154, 0.16781677434272257]
3
[1.61245154965971, 0.99950109600895498, 0.98817899950159394]
4
[2.0976176963403033, 0.99956104225878661, -1.0718698711283243]
5
[1.7320508075688772, 0.99950507875216699, nan]
6
[1.5491933384829668, 0.99940355715928153, 1.3619421046282456]
7
[1.8439088914585775, 0.99952465280469971, 0.99584888351341805]
8
[1.8439088914585775, 0.999505363097972, nan]
9
[1.7888543819998317, 0.99950626465120362, 1.0354907929844397]
10
[1.7320508075688772, 0.99951686233267822, 2.535177781630642]
11
[1.9493588689617927, 0.99949034909303747, 1.0152560966467397]
12
[1.6733200530681511, 0.99947668796809253, 0.998021069428361]
13
[1.61245154965971, 0.99943534894450914, -2.7894082447146489]
14
[1.7888543819998317, 0.99950651837122184, nan]
15
[1.7320508075688772, 0.99950354426611931, nan]
16
[1.7888543819998317, 0.99948797990356342, nan

In [280]:

tf.reset_default_graph()

In [214]:
communities = 2
group_size=5
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=1.0, p_out=0.0)).astype(np.float64) for i in range(1000)]


EigenDecomp_r_diff(10, 2, 5, data, 2, 3)

0
[2.0, 0.99846035320541238, nan]
1
[2.0, 0.99846035320541238, nan]
2
[2.0, 0.99846035320541238, nan]
3
[2.0, 0.99846035320541238, nan]
4
[2.0, 0.99846035320541238, nan]
5
[2.0, 0.99846035320541238, nan]
6
[2.0, 0.99846035320541238, nan]
7
[2.0, 0.99846035320541238, nan]
8
[2.0, 0.99846035320541238, nan]
9
[2.0, 0.99846035320541238, nan]


In [267]:
communities = 2
group_size=5
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.015, p_out=0.003)).astype(np.float64) for i in range(1000)]

answer = EigenDecomp_r_diff_proj(100, 2, 5, data, 1, 7, 2)

In [268]:
answer[:,2]

array([ 0.98559856,  1.        ,  0.94280904,  1.        ,  1.        ,
        1.        ,  0.98559856,  1.        ,  1.        ,  0.98559856,
        0.98559856,  0.94280904,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  0.9999896 ,  1.        ,
        1.        ,  1.        ,  0.98559856,  1.        ,  1.        ,
        0.98559856,  1.        ,  1.        ,  1.        ,  0.98559856,
        1.        ,  1.        ,  1.        ,  0.98620871,  0.98559856,
        1.        ,  1.        ,  1.        ,  0.98559856,  0.98559856,
        1.        ,  1.        ,  1.        ,  1.        ,  0.98559856,
        0.9999896 ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  0.98559856,  0.9999896 ,  0.94280904,
        0.94280904,  0.98559856,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.  

In [269]:

np.mean(np.abs(answer[:,2]))

0.98879436636310092

In [340]:
dim_graph
Adj

<tf.Tensor 'Placeholder_22:0' shape=<unknown> dtype=float64>

In [352]:
r = tf.cast(10^100000000000, dtype=tf.float64)
Adj = balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.4, p_out=0.1)
Bethe_Hesse_neg = (r*r-1)*tf.diag(tf.ones(shape=[communities*group_size], dtype=tf.float64))-tf.mul(r, Adj)
a = tf.self_adjoint_eig(Bethe_Hesse_neg)

In [358]:
with tf.Session() as sess:
    print sess.run([Bethe_Hesse_neg, a])
    

[array([[  1.00000000e+22,   0.00000000e+00,  -1.00000000e+11,
         -1.00000000e+11,   0.00000000e+00,   0.00000000e+00,
         -1.00000000e+11,  -1.00000000e+11,   0.00000000e+00,
         -1.00000000e+11,   0.00000000e+00,  -1.00000000e+11,
          0.00000000e+00,   0.00000000e+00,  -1.00000000e+11,
          0.00000000e+00,   0.00000000e+00,  -1.00000000e+11,
          0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+22,   0.00000000e+00,
         -1.00000000e+11,   0.00000000e+00,  -1.00000000e+11,
          0.00000000e+00,  -1.00000000e+11,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [ -1.00000000e+11,   0.00000000e+00,   1.00000000e+22,
          0.00000000e+00,   0.00000000e+00,  -1.00000000e+11,
         -1.00000000e+11,   0.00000000e+0